In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import KFold

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import Sampler

In [3]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "ctrp"  # Dataset{gdsc or ccle}


args = Args()
res, drug_feature, exprs, mut, cna, null_mask, pos_num = load_data(args)

In [4]:
epochs = []
true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)
device = "cpu"

for train_index, test_index in kfold.split(np.arange(pos_num)):
    sampler = Sampler(res, train_index, test_index, null_mask)
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    epochs.append(epoch)
    true_datas = pd.concat([true_datas, translate_result(true_data)])
    predict_datas = pd.concat([predict_datas, translate_result(predict_data)])

epoch:   0 loss:1.027443 auc:0.5500
epoch:  20 loss:0.403389 auc:0.9022
epoch:  40 loss:0.374367 auc:0.9081
epoch:  60 loss:0.369449 auc:0.9089
epoch:  80 loss:0.367819 auc:0.9092
epoch: 100 loss:0.367166 auc:0.9094
epoch: 120 loss:0.366684 auc:0.9095
epoch: 140 loss:0.366194 auc:0.9097
epoch: 160 loss:0.365586 auc:0.9099
epoch: 180 loss:0.364686 auc:0.9103
epoch: 200 loss:0.363051 auc:0.9110
epoch: 220 loss:0.359820 auc:0.9122
epoch: 240 loss:0.354923 auc:0.9139
epoch: 260 loss:0.349573 auc:0.9152
epoch: 280 loss:0.344322 auc:0.9169
epoch: 300 loss:0.336099 auc:0.9186
epoch: 320 loss:0.328972 auc:0.9200
epoch: 340 loss:0.322272 auc:0.9208
epoch: 360 loss:0.314582 auc:0.9220
epoch: 380 loss:0.308923 auc:0.9245
epoch: 400 loss:0.306963 auc:0.9246
epoch: 420 loss:0.296360 auc:0.9268
epoch: 440 loss:0.290002 auc:0.9286
epoch: 460 loss:0.284708 auc:0.9302
epoch: 480 loss:0.280221 auc:0.9315
epoch: 500 loss:0.277666 auc:0.9319
epoch: 520 loss:0.273540 auc:0.9334
epoch: 540 loss:0.270945 auc

In [5]:
true_datas.reset_index(drop=True).to_csv("true_ctrp.csv")
predict_datas.reset_index(drop=True).to_csv("pred_ctrp.csv")